## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")


In [6]:
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Lukulu,ZM,-14.3707,23.2420,70.25,65,65,4.38,broken clouds
1,1,Butembo,CD,0.1500,29.2833,59.00,99,90,1.43,light rain
2,2,Mataura,NZ,-46.1927,168.8643,50.49,84,13,6.11,few clouds
3,3,Albany,US,42.6001,-73.9662,66.24,34,100,5.99,overcast clouds
4,4,Avarua,CK,-21.2078,-159.7750,71.65,100,85,3.44,overcast clouds
5,5,Tura,IN,25.5198,90.2201,75.07,53,0,5.70,clear sky
6,6,Rikitea,PF,-23.1203,-134.9692,75.33,78,100,12.03,overcast clouds
7,7,Yar-Sale,RU,66.8333,70.8333,15.55,95,54,7.76,broken clouds
8,8,Tiksi,RU,71.6872,128.8694,-6.23,100,98,3.13,overcast clouds
9,9,Hilo,US,19.7297,-155.0900,72.28,93,75,4.61,broken clouds


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                      (city_data_df["Max Temp"] >= min_temp)]

In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                230
City                   230
Country                228
Lat                    230
Lng                    230
Max Temp               230
Humidity               230
Cloudiness             230
Wind Speed             230
Current Description    230
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                228
City                   228
Country                228
Lat                    228
Lng                    228
Max Temp               228
Humidity               228
Cloudiness             228
Wind Speed             228
Current Description    228
dtype: int64

In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Tura,IN,75.07,clear sky,25.5198,90.2201,
6,Rikitea,PF,75.33,overcast clouds,-23.1203,-134.9692,
15,Atuona,PF,78.76,scattered clouds,-9.8000,-139.0333,
18,Cabo San Lucas,MX,85.30,clear sky,22.8909,-109.9124,
22,Sambava,MG,76.75,light rain,-14.2667,50.1667,
28,Inisa,NG,88.54,broken clouds,7.8500,4.3333,
31,Tuy Hoa,VN,76.24,few clouds,13.0833,109.3000,
38,Namatanai,PG,78.64,light rain,-3.6667,152.4333,
43,Caborca,MX,80.92,clear sky,30.5838,-111.1010,
44,Fuerte,PH,82.26,scattered clouds,17.5381,120.3644,


In [16]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
   
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df = hotel_df.replace("", nan_value)

hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Tura,IN,75.07,clear sky,25.5198,90.2201,Hotel RIKMAN Continental
6,Rikitea,PF,75.33,overcast clouds,-23.1203,-134.9692,People ThankYou
15,Atuona,PF,78.76,scattered clouds,-9.8000,-139.0333,Villa Enata
18,Cabo San Lucas,MX,85.30,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
22,Sambava,MG,76.75,light rain,-14.2667,50.1667,Melrose
...,...,...,...,...,...,...,...
710,Prainha,BR,85.44,broken clouds,-1.8000,-53.4800,Casa Rocha
716,Buenos Aires,AR,78.01,clear sky,-34.6132,-58.3772,Hotel NH Buenos Aires City
718,Tefe,BR,82.51,scattered clouds,-3.3542,-64.7114,Aracati Hotel
719,Sao Miguel Do Araguaia,BR,88.95,broken clouds,-13.2750,-50.1628,Hotel Executivo Palace


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))